In [1]:
ls

Vectorizing Diversity Score.ipynb  reranking.ipynb
diversity_score.py


In [2]:
import pandas as pd
import numpy as np
import time
from scipy.spatial.distance import pdist

embedding_cols = [
#     'year',
    'acousticness',
    'danceability',
    'duration_ms',
    'energy',
    'instrumentalness',
    'key',
    'liveness',
    'loudness',
    'mode',
    'speechiness',
    'tempo',
    'time_signature',
    'valence'
]
msd = pd.read_hdf('../data/full_msd_with_audio_features.h5', key='df')
msd = msd[['song_id'] + embedding_cols]
msd.head()

def get_MUSIC(song_ids):
    song_vectors = msd.loc[msd['song_id'].isin(song_ids)][list(feature_MUSIC_dict.keys())].values
    return np.sum(np.dot(song_vectors, feature_MUSIC_matrix), axis=0)/len(song_ids)

def get_list_dissimilarity(song_ids):
    song_vectors = msd.loc[msd['song_id'].isin(song_ids)][embedding_cols].values
    return np.mean(pdist(song_vectors, 'cosine'))

song_ids = list(msd.head(1000).song_id)
start = time.time()
print(f'Dissim was {get_list_dissimilarity(song_ids)}')
print(f'It took {time.time() - start}s')

Dissim was 1.554302328841804e-07
It took 0.05302000045776367s


In [3]:
msd.shape

(447499, 14)

In [4]:
# NOTE: if song_ids are passed in as normal strings, replace instances of song_id with prepare_song_id(song_id)

def prepare_song_id(song_id):
  return('b\'' + song_id + '\'')


def get_diversity(song_vector_list):
  dissim = 0
  n = len(song_vector_list)
  for i in range(n):
    for j in range(n):
      dissim += np.linalg.norm(song_vector_list[i] - song_vector_list[j])

  return dissim / ((n / 2) * (n - 1))


def convert_recs_to_embeddings(song_ids):
  embeddings_list = []

  for song_id in song_ids:
    # NOTE: if song_ids are passed in as normal strings, replace all following instances of song_id with prepare_song_id(song_id)
    if (song_id in msd.song_id.values):
      row = msd.loc[msd['song_id'] == song_id]
      embedding = row.iloc[:, 12:]
#       print(embedding)
      embedding = np.array(embedding.values.tolist()[0])
      embeddings_list.append(embedding)

  return embeddings_list


def get_diversity_score(song_ids):
  # convert to list of embeddings
  embedding_list = convert_recs_to_embeddings(song_ids)
#   embedding_list = msd.loc[msd['song_id'].isin(song_ids)][embedding_cols].values
  # calculate diversity
  return get_diversity(embedding_list)


song_ids = list(msd.head(1000).song_id)
start = time.time()
print(f'Dissim was {get_diversity_score(song_ids)}')
print(f'It took {time.time() - start}s')

Dissim was 1.0264996880777573
It took 58.21383213996887s


In [5]:
x = np.ones((3,3))
pdist(x)
# pdist([[1,2,3], [1,2,3]])

array([0., 0., 0.])